# test

In [1]:
!nvidia-smi

Wed Jun 12 10:32:43 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:01:00.0  On |                  N/A |
| 38%   54C    P2    33W / 250W |   1474MiB / 12033MiB |      3%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   48C    P8     9W / 215W |      2MiB /  8119MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [2]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Apr_24_19:10:27_PDT_2019
Cuda compilation tools, release 10.1, V10.1.168


In [3]:
!pgc++ -V


pgc++ 19.4-0 LLVM 64-bit target on x86-64 Linux -tp haswell 
PGI Compilers and Tools
Copyright (c) 2019, NVIDIA CORPORATION.  All rights reserved.


# Normal
[normal_C.cpp](/edit/normal_C.cpp)

In [4]:
!nvcc --run normal_C.cpp -O2 -o normal_C

No input given.
Running with N = 1024

Running Normal C: 1285.17 ms


In [5]:
!./normal_C 128

Running with N = 128

Running Normal C: 1.47 ms


Try
Changing -O3 to -O0
Changing input paramater `./normal_C 128`. Higher than default, 1024, will take longer.

# OpenMP
[openmp.cpp](/edit/openmp.cpp)

In [6]:
!nvcc --run -ccbin pgc++ -O2 -Xcompiler "-V19.4 -mp" openmp.cpp -o openmp

No input given.
Running with N = 1024

Running Normal C: 1521.29 ms
Running OpenMP: 243.20 ms
Test passed.


Try
Removing -mp from build command
Changing omp_set_num_threads( 6 ) to 2, 4, or more.
Changing input paramater `./normal_C 128`. Higher than default, 1024, will take longer.

# OpenACC
[openacc.cpp](/edit/openacc.cpp)

In [7]:
!nvcc --run -ccbin pgc++ -O2 -Xcompiler "-V19.4 -Bstatic_pgi -acc -ta=tesla:nordc" openacc.cpp -o openacc

No input given.
Running with N = 1024

Running Normal C: 1320.65 ms
Running OpenACC: Test passed.


No time??

In [8]:
!nvcc --run -ccbin pgc++ -O2 -Xcompiler "-V19.4 -Bstatic_pgi -acc -ta=tesla:nordc -ta=time" openacc.cpp -o openacc

No input given.
Running with N = 1024

Running Normal C: 1305.89 ms
Running OpenACC: CUPTI ERROR: cuptiActivityEnable(CUPTI_ACTIVITY_KIND_KERNEL) returned: CUPTI_ERROR_INSUFFICIENT_PRIVILEGES, 
	 at ../src-cupti/prof_cuda_cupti.c:265.
Test passed.

Accelerator Kernel Timing data
/home/belt/git/computeWorks_examples/computeWorks_mm/jupyter/openacc.cpp
  _Z7openACCifPKfS0_fPfRKi  NVIDIA  devicenum=0
    time(us): 11,381
    43: compute region reached 5 times
        49: kernel launched 5 times
            grid: [8x1024]  block: [128]
            elapsed time(us): total=10,907 max=2,211 min=2,121 avg=2,181
    43: data region reached 10 times
        43: data copyin transfers: 10
             device time(us): total=7,367 max=1,091 min=648 avg=736
        58: data copyout transfers: 5
             device time(us): total=4,014 max=1,082 min=641 avg=802


Why error? While if you are running the newest driver, 410.67, profiling requires elevated permissions

In [ ]:

!sudo LD_LIBRARY_PATH=/usr/local/cuda/extra/CUPTI/lib64:$LD_LIBRARY_PATH ./openacc

[sudo] password for belt: 

# BLAS
[blas.cpp](/edit/blas.cpp)

In [ ]:
!nvcc --run -ccbin pgc++ -O2 -Xlinker "-lblas" -Xcompiler "-V19.4" blas.cpp -o blas

In [ ]:
!./blas 128

# cuBLAS
[cublas.cpp](/edit/cublas.cpp)

In [ ]:
!nvcc --run -O2 -lcublas cublas.cpp -o cublas

In [ ]:
!./cublas 128

# CUDA
[cuda.cu](/edit/cuda.cu)

In [ ]:
!nvcc --run -O2 cuda.cu -o cuda

In [ ]:
!./cuda 128